Notebook for cropping, slicing and other data prep operations on ground truth, initial segmentation and image data for error correction pipeline

# Ground truth

We slice the ground truth into blocks of the same size as the predicted segmentation. The extents of the ground truth are also cropped to the volume containing all the labeled objects. Parameters for these slicing and cropping operations are globally defined in `data_locs.json`

In [4]:
# Code to generate 48 nm downsampled version of GT and find bounding box of labeled volume

# gt8nm_file = data_locs["gt"]["dir"] + data_locs["gt"]["8nm"]
# find global bbox of GT segmentation
# from skimage import measure
# use_global_bbox = True # set to false if you want blockwise bounding boxes
#############################
# loads entire GT - caution!
# this should ideally be a one time evaluation
# comment enclosed code and manually set extents below 
# if global bounding box is known
# and if downsampled version is already on disk
# gt48nm_full = read3d_h5(gt8nm_file, 'main', dsmpl=(6,6,1))
# gt48nm_full = np.swapaxes(gt48nm_full,1,2)
# gt48nm_full = np.swapaxes(gt48nm_full,0,1)
# fout = data_locs["gt"]["dir"] + "gt48nm_pf-align1_1k.h5"
# writeh5(fout, 'main', gt48nm_full, compression="gzip", 
#         chunks=(1,gt48nm_full.shape[1],gt48nm_full.shape[2]))
# gt_mask = np.zeros_like(gt48nm_full)
# gt_mask[gt48nm_full>0] = 1
# del gt48nm_full
# props = measure.regionprops(gt_mask, cache=True)
# del gt_mask
# bbox = props[0].bbox
# print bbox
#############################
# bbox = (0, 16, 208, 1001, 556, 696) # save these co-ordinates to data_locs.json

In [1]:
import json
from cerebellum.utils.seg_prep import *

with open('data_locs.json') as f:
	data_locs = json.load(f)

resolution = (30, 48, 48)
gt48nm_file = data_locs["gt"]["dir"] + data_locs["gt"]["48nm"]
bbox = data_locs["gt"]["48nm-bbox"]
block_indices = range(2)

for i in block_indices:
    print "Generating block %d from"%(i)
    print gt48nm_file
    zz = i*data_locs["block-size"]+data_locs["aff-offset"]
    gt_block = SegPrep("gt%04d"%(zz), resolution)
    block_lims = ((data_locs["block-size"]*i,data_locs["block-size"]*(i+1)),
                  (bbox[1],bbox[4]),(bbox[2],bbox[5]))
    print block_lims
    gt_block.read(gt48nm_file, "main", block_lims=block_lims)
    print gt_block.shape
    gt_block.relabel()
    gt_block.write()

/home/srujanm/anaconda2/envs/em/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Generating block 0 from
/home/srujanm/cerebellum/data/gt48nm_pf-align1_1k.h5
((0, 90), (16, 556), (208, 696))
[90, 540, 488]
Generating block 1 from
/home/srujanm/cerebellum/data/gt48nm_pf-align1_1k.h5
((90, 180), (16, 556), (208, 696))
[90, 540, 488]


# PF segmentation

The predicted segmentation needs to be offset and padded with zeros to overlap with the GT

In [2]:
import json
from cerebellum.utils.seg_prep import *

with open('data_locs.json') as f:
	data_locs = json.load(f)

resolution = (30, 48, 48)
dsmpl = (1,6,6)
offset = data_locs["initial-seg"]["8nm-offset"] # offset aligns segmentation with GT
pf8nm_file = data_locs["initial-seg"]["dir"] + data_locs["initial-seg"]["8nm-pf-linked"]

block_indices = range(2)

for i in block_indices:
    print "Generating block %d from"%(i)
    zz = i*data_locs["block-size"]+data_locs["aff-offset"]
    if zz!=data_locs["aff-offset"]: # adjust block index
        pf8nm_file = pf8nm_file[:-7]+"%04d.h5"%(zz)
    print pf8nm_file
    pf_block = SegPrep("pred-pf-crop2gt-%04d"%(zz), resolution)
    pf_block_lims = ((0,data_locs["block-size"]),
                   (max(0,dsmpl[1]*bbox[1]+offset[1]),dsmpl[1]*bbox[4]+offset[1]),
                   (max(0,dsmpl[2]*bbox[2]+offset[2]),dsmpl[2]*bbox[5]+offset[2]))
    print pf_block_lims
    pf_block.read(pf8nm_file, "main", dsmpl=dsmpl, block_lims=pf_block_lims)
    print pf_block.shape
    pf_block.padzeros((gt_block.shape[0],gt_block.shape[1]-pf_block.shape[1],gt_block.shape[2]), 1)
    pf_block.relabel()
    pf_block.write()

Generating block 0 from
/mnt/hp03/donglai/public/cere_pf/data_link_strict/0014.h5
((0, 90), (52, 3292), (1204, 4132))
[90, 533, 488]
Generating block 1 from
/mnt/hp03/donglai/public/cere_pf/data_link_strict/0104.h5
((0, 90), (52, 3292), (1204, 4132))
[90, 533, 488]


# Image

In [18]:
# generate and save cropped version of image
im48nm_file = data_locs["image"]["dir"] + data_locs["image"]["48nm"]
im48nm_block = read3d_h5(im48nm_file, 'main', block_lims=block_lims)
print im48nm_block.shape
fout = "/home/srujanm/cerebellum/data/im48nm_cropped_%04d.h5"%(zz)
writeh5(fout, 'main', im48nm_block, compression="gzip", chunks=(1,im48nm_block.shape[1],im48nm_block.shape[2]))

(90, 540, 488)


In [19]:
# repeat for 16 nm res version
im16nm_file = data_locs["image"]["dir"] + data_locs["image"]["16nm"]
if zz!=14:
     im16nm_file = im16nm_file[:-7]+"%04d.h5"%(zz)
im_block_lims = ((0,data_locs["block-size"]),
                (3*block_lims[1][0],3*block_lims[1][1]),
                (3*block_lims[2][0],3*block_lims[2][1]))
im16nm_block = read3d_h5(im16nm_file, 'main', block_lims=im_block_lims)
print im16nm_block.shape

(90, 1620, 1464)


In [20]:
fout = "/home/srujanm/cerebellum/data/im16nm_cropped_%04d.h5"%(zz)
writeh5(fout, 'main', im16nm_block, compression="gzip", chunks=(1,im16nm_block.shape[1],im16nm_block.shape[2]))

# Check alignment

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

im_final = read3d_h5('/home/srujanm/cerebellum/data/im48nm_cropped_%04d.h5'%(zz), 'main')
gt_final = read3d_h5('/home/srujanm/cerebellum/data/gt48nm_cropped_pf_%04d.h5'%(zz), 'main')
seg_final = read3d_h5('/home/srujanm/cerebellum/data/seg48nm_cropped_pf_%04d.h5'%(zz), 'main')

In [ ]:
# Pick two IDs corresponding to the same object from neuroglancer
gt_id = 793
seg_id = 4012
# Plot a section to see if alignment is grossly off
zslice = 41
mask1 = gt_final[zslice,100:150,160:220]==gt_id
mask2 = seg_final[zslice,100:150,160:220]==seg_id
mismatch = mask1^mask2
fig, (ax1, ax2, ax3) = plt.subplots(3,1,figsize=(15,15))
ax1.imshow(mask1)
ax2.imshow(mask2)
ax3.imshow(mismatch)

In [ ]:
def relabel(seg):
    """
    Relabels a segmentation such that max ID = # objects
    Args:
        seg (ndarray): 3D segmentation
    Returns:
        seg_relabeled (ndarray)
    """
    seg_ids = get_vols(seg)[0]
    n_ids = len(seg_ids)
    max_id = np.max(seg_ids)
    if max_id == n_ids:
        return seg
    missing_ids = np.sort(np.array(list(set(range(max_id+1)).difference(set(seg_ids)))))
    seg_relabel = seg
    for i in range(len(missing_ids)):
        if i==len(missing_ids)-1:
            ids_to_correct = range(missing_ids[i]+1, max_id+1)
        else:
            ids_to_correct = range(missing_ids[i]+1, missing_ids[i+1])
        for j in ids_to_correct:
            seg_relabel[seg==j] = j-(i+1) #TODO (Jeff): speed this up using object-wise bounding boxes
    return seg_relabel

In [ ]:
seg_calc = 
seg_corrected = relabel(seg_calc)
print np.max(seg_corrected), len(get_vols(seg_corrected)[0])

In [ ]:
fout = data_locs["trials"] + "gt48nm_cropped_relabeled_pf_0014.h5"
writeh5(fout, 'main', seg_calc, compression="gzip", chunks=(1,im16nm_block.shape[1],im16nm_block.shape[2]))